In [12]:
import pandas as pd
import numpy as np
import os, os.path
import pathlib
import json
import re
from functools import partial
from datetime import date
from statistics import mean
import collections
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
# zbiorcza ramka do głownej analizy
merged_message = pd.DataFrame(
    {"file" : [],
    "participants" : [],
     "mean_my_res_time" : [],
     "mean_other_res_time" :[],
    "my_messages" : [],
     "all_messages" : []
    },
    )

merged_common_words = pd.DataFrame(
    {"word" : [],
    "count" : [],
     "file" : [],
    },
    )

merged_time = pd.DataFrame(
    {"date" : [],
     "file" : []
    },
    )


In [14]:
# aby emotki działały
fix_mojibake_escapes = partial(
        re.compile(rb'\\u00([\da-f]{2})').sub,
        lambda m: bytes.fromhex(m[1].decode()),
    )

In [35]:
#stopwords = set(line.strip() for line in open('stopwords.txt', 'utf8'))
stopwords = set(line.strip() for line in open('stopwords.txt', encoding='utf8'))

In [36]:
        me = "Łukasz Grabarski"

In [31]:
    #idziemy po wzystkich jsonach w folderze
for file in pathlib.Path('inbox').glob('**/*.json'):
        #file = "inbox/agnieszkagrabarska_ubdmm4mxea/message_1.json"
        #rozmowa = "Agnieszka Grabarska"
        rozmowa = os.path.basename(file.parent).split('_')[0]    

        #importujemy plik z działającymi polskimi znakami
        with open(file, 'rb') as binary_data:
            repaired = fix_mojibake_escapes(binary_data.read())
        data = json.loads(repaired)

        #normalna eksploracja danych
        df_participants = pd.DataFrame(data['participants'])
        df_messages = pd.DataFrame(data['messages'])
        df_messages['date'] = pd.to_datetime(df_messages.timestamp_ms, unit = 'ms')

        participants = df_participants['name'].size
        mean_other_res_time = -1
        mean_my_res_time = -1
        my_messages = len(df_messages[df_messages.sender_name == me])
        all_messages = len(df_messages['sender_name'])
        title = data['title']

        #if len(df_messages['sender_name']) < 2:
            #continue
        #badamy czas odpowiedzi
        if participants == 2:
            senders = list(df_messages['sender_name'])
            times =  list(df_messages['timestamp_ms'])
            times.reverse()
            senders.reverse()
            response_time_my = list()
            response_time_other = list()
            iprev = 0

            for i in range(len(times)):
                if senders[i] != senders[iprev]:
                    if senders[i] == me:
                        response_time_my.append(times[i] - times[iprev])
                    else:
                        response_time_other.append(times[i] - times[iprev])

                iprev = i

            if len(response_time_my) > 2:          
                mean_my_res_time = round(mean(response_time_my))
            if len(response_time_other) > 2:          
                mean_other_res_time = round(mean(response_time_other))

        # najczęstrze słowa
        #print(df_messages.query('sender_name == @me'))
        
        print(title)
        if 'content' not in df_messages:
            continue
        
        content = list(df_messages.query('sender_name == @me')['content'])
        content = ' '.join(map(str, content))
        wordcount = {}

        for word in content.lower().split():
            word = word.replace(".","")
            word = word.replace(",","")
            word = word.replace("nan","")
            word = word.replace("-"," ")
            word = word.replace(":","")
            word = word.replace("\"","")
            word = word.replace("!","")
            word = word.replace("*","")
            if word not in stopwords:
                if word not in wordcount:
                    wordcount[word] = 1
                else:
                    wordcount[word] += 1

        n_print = 10
        word_counter = collections.Counter(wordcount)
        
        if word_counter.get("") != None:
            word_counter.pop("")
        word_counter = (pd.DataFrame(list(word_counter.most_common())))
        word_counter['file'] = title
        word_counter.rename(columns = {0 : 'word', 1 : 'count'}, inplace = True)    

        #czas wysłania wiadomości

        messages_time = pd.DataFrame(df_messages.query('sender_name == @me')['date'])
        #messages_time.rename(columns = {0 : 'date'}, inplace = True)
        messages_time['file'] = title

       # print(messages_time)


       # print(word_counter.most_common(n_print))

       # for word, count in word_counter.most_common(n_print):
        #    print(word, ": ", count)


        #mergujemy z główną ramką
        df_to_merge = pd.DataFrame({"file" : title,
                                    "participants" : participants,
                                    "mean_my_res_time" : mean_my_res_time,
                                    "mean_other_res_time" : mean_other_res_time,
                                    "my_messages" : my_messages,
                                    "all_messages" : all_messages},
                                  index=[1])

        #print(df_to_merge)
        merged_message = pd.concat([merged_message, df_to_merge])
        merged_common_words = pd.concat([merged_common_words, word_counter])
        merged_time = pd.concat([merged_time, messages_time])

10.11
18 Łukasza🎉
18
19.03 🥳
20 urodzinki tymka
3B Chłopacy
3B eWychowawcza
3D
3D
4min4life <3
8 LO MINECRAFT IP: grzybek.bymarcin.com:2137
8melowa Grupa niePoprawnych Politycznie
8melo
9 lutego ✨
Adam Adam
Adam Ciborek
Adam Dęby
Adam, Jakub, Kacper i 11 innych użytkowników
Adam Kąkolewski
Adam Krystek
Adam Naumiec
Adam Przemysław Chojecki
Adrian Cieśla
Adrian Dolecki
Adrian Gołębiowski
Adrian Kaźmierczuk
Adrian Zaręba
Aga Mańke
Aga Mańke
Aga Mańke
Aga Mańke
Aga Mańke
Aga Mańke
Aga Mańke
Agata Andrzejczak
Agata Boruta
Agata Chałupniczak
Agata Gerl
Agata i Zosia
Agata Kaczmarek
Agata Makarewicz
Agata, Szy, Zosia i 3 innych użytkowników
Agnieszka Balcerowiak
Agnieszka Dziadek
Agnieszka Grabarska
Agnieszka Kaczmarek
Agnieszka Kaczmarek
Agnieszka Zwolińska
AJR
AKCJA: WARSZAFKA
Alan Kirpa
Ala Röhl
Ala Tacakiewicz
Alek Malinowski
Aleksander Mieszała
Aleksandra Drozd
Aleksandra Dworzak
Aleksandra Grabarska
Aleksandra i Magdalena
Aleksandra Konieczka
Aleksandra Kulczycka
Aleksandra Ortyl
Aleks

Mikołaj Dziewiatowski
Mikołaj i Aga
Mikołaj Majchrzak
Mikołaj Miedziak
Mikołaj Mróz
Mikołaj Popek
Mikołaj Szawerda
Mikołaj Szawerda
Mikołaj Szawerda
Mikołaj Szawerda
Mikołaj Szumlański
Mikołaj Wieczorek
Milionerzy TVN
Miłość Jest Piękna
Miłość Jest Piękna
Miłość Jest Piękna
Miłość Jest Piękna
Miłość Jest Piękna
Miłość Jest Piękna
Miłość Jest Piękna
Miłość Jest Piękna
Miłość Jest Piękna
Miłość Jest Piękna
Miłość Jest Piękna
Miłość
Miłosz Wiertel
Mimka Halinka
Mir Ban
misja kebsik😎💞💕
Misja projekt 😈
MKA ✌️
MKA ✌️
MKA ✌️
MKA ✌️
MKA ✌️
Mobzilla
Monika Gruse-Chrzanowska
Monika Ignaszewska
MRMP IV Kadencja
MRM po godzinach
Nadia Kluz
Najlepsi Z Najlepszych 󾌴󾬒󾬐
Natalia Jankowska
Natalia Kuchta
Natalia Niedźwiedzka
Natalia Safiejko
Natalia Wieczorek
Niedziela Wieczur Humor Gituwa
Nie Głodowanie Razem
Niemiecki 3B Grupa Mniej Zaawansowana
NIE MÓWMY O TYMXD
Niespodzianka dla Amelii
Night Hunting
Nikodem Dudek
Nocki u Zosi 💖
Norbert Posadzinski
Notatki na Polski
Obchody 10. rocznicy Libacji na sk

In [38]:
#df['date'] = pd.to_datetime(df.timestamp_ms, unit='ms') <- linijka która zamienia ten dziwny czas timestamp na normlaną datę
merged_message.to_csv(me + '_'+ 'merged_message.csv')
merged_common_words.to_csv(me + '_'+ 'merged_common_words.csv')
merged_time.to_csv(me + '_'+ 'merged_time.csv')


,date,file
0,2022-12-06 09:26:13.503000,Agnieszka Grabarska
2,2022-12-06 09:20:13.788000,Agnieszka Grabarska
3,2022-12-06 06:40:18.386000,Agnieszka Grabarska
7,2022-12-04 20:55:44.016000,Agnieszka Grabarska
9,2022-12-04 20:44:24.122000,Agnieszka Grabarska
...,...,...
125,2020-11-19 13:58:17.789000,Zuza Rybacka
126,2020-11-19 13:58:15.844000,Zuza Rybacka
129,2020-11-19 13:57:39.465000,Zuza Rybacka
130,2020-11-19 13:56:05.094000,Zuza Rybacka


,word,count,file
0,dobranoc,84.0,Agnieszka Grabarska
1,kocham,42.0,Agnieszka Grabarska
2,chyba,38.0,Agnieszka Grabarska
3,😂,33.0,Agnieszka Grabarska
4,ok,31.0,Agnieszka Grabarska
...,...,...,...
189,konkretnie,1.0,Zuza Rybacka
190,kupił,1.0,Zuza Rybacka
191,niedługo,1.0,Zuza Rybacka
192,informacja,1.0,Zuza Rybacka
